In [5]:
from dotenv import load_dotenv
load_dotenv()
import kaggle

kaggle.api.authenticate()
kaggle.api.dataset_download_files('awsaf49/cbis-ddsm-breast-cancer-image-dataset', path='data', unzip=True)



Dataset URL: https://www.kaggle.com/datasets/awsaf49/cbis-ddsm-breast-cancer-image-dataset


In [ ]:
def general_data_clearing(df):
    # droping rows with missing values
    df.dropna(inplace=True)

    # spilting image paths into separate columns so I can find the corresponding image file easily
    df = path_spilting(df, 'image file path','full')
    df = path_spilting(df, 'cropped image file path','cropped')
    df = path_spilting(df, 'ROI mask file path','mask')

    # replacing spaces in column names to underscores
    df.columns = df.columns.str.replace(' ', '_')

    df.loc{df['pathology']}

    return df


def path_spilting(df, col_name,image_type):
    new = df[col_name].str.split('/',n =3,expand=True)
    df[f'image_folder_{image_type}']=new[2]
    if image_type == 'cropped':
        df[f'image_name_{image_type}']=new[3]
        df[f'image_name_{image_type}'] = df[f'image_name_{image_type}'].str.replace('.dcm\n', '')

    df.drop(columns=[col_name], inplace=True)
    return df

: 

In [ ]:
import pandas as pd

image_info_df_train = pd.read_csv('data\csv\calc_case_description_train_set.csv')

image_info_df_test = pd.read_csv('data\csv\calc_case_description_test_set.csv')

general_data_clearing(image_info_df_train)

image_info_df_train.head()

,patient_id,breast_density,left_or_right_breast,image_view,abnormality_id,abnormality_type,calc_type,calc_distribution,assessment,pathology,subtlety,image_folder_full,image_folder_cropped,image_name_cropped,image_folder_mask
0,P_00005,3,RIGHT,CC,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,1.3.6.1.4.1.9590.100.1.2.474143160103683865197...,1.3.6.1.4.1.9590.100.1.2.393344010211719049419...,000001,1.3.6.1.4.1.9590.100.1.2.393344010211719049419...
1,P_00005,3,RIGHT,MLO,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,1.3.6.1.4.1.9590.100.1.2.250596608311207922527...,1.3.6.1.4.1.9590.100.1.2.296281207812130400303...,000001,1.3.6.1.4.1.9590.100.1.2.296281207812130400303...
2,P_00007,4,LEFT,CC,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,1.3.6.1.4.1.9590.100.1.2.228699627313487111012...,1.3.6.1.4.1.9590.100.1.2.314135871111943890422...,000001,1.3.6.1.4.1.9590.100.1.2.314135871111943890422...
3,P_00007,4,LEFT,MLO,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,1.3.6.1.4.1.9590.100.1.2.104743410411133110629...,1.3.6.1.4.1.9590.100.1.2.914582796124855152034...,000001,1.3.6.1.4.1.9590.100.1.2.914582796124855152034...
20,P_00010,3,LEFT,CC,1,calcification,ROUND_AND_REGULAR-LUCENT_CENTER-DYSTROPHIC,DIFFUSELY_SCATTERED,2,BENIGN_WITHOUT_CALLBACK,4,1.3.6.1.4.1.9590.100.1.2.271665327610523966032...,1.3.6.1.4.1.9590.100.1.2.134997264213461213722...,000001,1.3.6.1.4.1.9590.100.1.2.134997264213461213722...


In [ ]:
from sklearn.preprocessing import OneHotEncoder



image_info_df_train=pd.get_dummies(image_info_df_train, columns=['left_or_right_breast','image_view','abnormality_type','calc_type','calc_distribution']

Data Importing and file type conversion

In [8]:
# Ensuring images counts in folders match counts from csv

import os
def count_files(folder_path):
    count = 0
    # Iterate directory
    for path in os.listdir(folder_path):
        # check if current path is a file
        if os.path.isfile(os.path.join(folder_path, path)):
            count += 1

    return count

print(f"Normal DCM : {count_files('data/DCM/normal')}")
print(f"Cancer DCM : {count_files('data/DCM/cancer')}")

Normal DCM : 5276
Cancer DCM : 27


In [ ]:
# Converting from .dcm to .PNG

import pydicom as dicom
import matplotlib.pyplot as plt
import os
import cv2
import PIL
import pandas as pd
import csv


# Specify the .dcm folder path
cancer_folder_path = r"data\DCM\cancer"
normal_folder_path = r"data\DCM\normal"
# Specify the .png folder path
cancer_PNG_folder_path = r"data\PNG\cancer"
normal_PNG_folder_path = r"data\PNG\normal"

def dcm_to_PNG(dcm_folder, PNG_folder):
    images_path = os.listdir(dcm_folder)
    for n, image in enumerate(images_path):
        try:
                
            ds = dicom.dcmread(os.path.join(dcm_folder, image))
            pixel_array_numpy = ds.pixel_array
            image = image.replace('.dcm', '.png')

            if os.path.isfile(os.path.join(PNG_folder, image)):
                print('Image {} already exists. Skipping...'.format(image))
                continue

            cv2.imwrite(os.path.join(PNG_folder, image), pixel_array_numpy)
            if n % 50 == 0:
                print('{} image converted'.format(n))
        except:
            print('Error occurred while reading or writing the image:', image)
            continue
    return None

dcm_to_PNG(cancer_folder_path, cancer_PNG_folder_path)
dcm_to_PNG(normal_folder_path, normal_PNG_folder_path)

Image 109361654.png already exists. Skipping...
Image 1168039233.png already exists. Skipping...
Image 1296917290.png already exists. Skipping...
Image 1349529516.png already exists. Skipping...
Image 1416534699.png already exists. Skipping...
Image 1434430689.png already exists. Skipping...
Image 1497446939.png already exists. Skipping...
Image 1576775200.png already exists. Skipping...
Image 1581726224.png already exists. Skipping...
Image 1639342205.png already exists. Skipping...
Image 1645279252.png already exists. Skipping...
Image 1710665236.png already exists. Skipping...
Image 1873677554.png already exists. Skipping...
Image 1925785468.png already exists. Skipping...
Image 1997115272.png already exists. Skipping...
Image 2000410429.png already exists. Skipping...
Image 2017526248.png already exists. Skipping...
Image 2025746473.png already exists. Skipping...
Image 2047450417.png already exists. Skipping...
Image 2102657462.png already exists. Skipping...
Image 2141709197.png 

In [ ]:
# Checking the converted images counts
# Some images were not converted due to errors or file formats not supported
print(f"Normal PNG : {count_files('data/PNG/normal')}")
print(f"Cancer PNG : {count_files('data/PNG/cancer')}")

Normal File count: 3053
Normal File count: 27
